In [9]:
# First Task Lab 5
class Queue:
    def __init__(self):
        self.items = []

    def insert(self, value):
        self.items.append(value)

    def pop(self):
        if not self.is_empty():
            return self.items.pop(0)
        else:
            print("empty queue")
            return None

    def is_empty(self):
        return len(self.items) == 0

q = Queue()
q.insert(1)
q.insert(2)
print(q.pop())  
print(q.pop())  
print(q.is_empty())  

1
2
True


In [18]:
# Second Task Lab 5
import json

class Queue:
    queues = {}

    @classmethod
    def save(cls, filename):
        with open(filename, 'w') as file:
            queue_list = []
            for name, queue in cls.queues.items():
                queue_data = {'name': queue.name, 'size': queue.size, 'items': queue.items}
                queue_list.append(queue_data)
            json.dump(queue_list, file)

    @classmethod
    def load(cls, filename):
        with open(filename, 'r') as file:
            queue_list = json.load(file)
            for queue_data in queue_list:
                name = queue_data['name']
                size = queue_data['size']
                items = queue_data['items']
                cls.queues[name] = cls(name, size)
                cls.queues[name].items = items

    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.items = []
        self.__class__.queues[name] = self

    def insert(self, value):
        if len(self.items) < self.size:
            self.items.append(value)
        else:
            raise QueueOutOfRangeException("Queue size exceeded")

    def pop(self):
        if self.is_empty():
            print("empty queue")
            return None
        return self.items.pop(0)

    def is_empty(self):
        return len(self.items) == 0


class QueueOutOfRangeException(Exception):
    def __init__(self, message="Queue size exceeded"):
        self.message = message
        super().__init__(self.message)

In [19]:
q1 = Queue("queue1", 3)
q1.insert(1)
q1.insert(2)
q1.insert(3)
# q1.insert(4)

q2 = Queue("queue2", 3)
q2.insert(10)
q2.insert(20)
q2.insert(30)

Queue.save("queues.json")
Queue.load("queues.json")

q_loaded = Queue.queues["queue1"]
print(q_loaded.pop())
q_loaded = Queue.queues["queue2"]
print(q_loaded.pop())

1
10


In [10]:
# Second Task Lab 5
class QueueWithAttributes:
    all_queues = {}

    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.items = []

        if self.name in self.all_queues:
            raise ValueError("Queue with this name already exists.")
        else:
            self.all_queues[self.name] = self

    def insert(self, value):
        """Inserts a new value at the rear of the queue."""
        if len(self.items) < self.size:
            self.items.append(value)
        else:
            raise QueueOutOfRangeException("Queue size exceeded.")

    def pop(self):
        """Returns and removes a value from the front of the queue."""
        if not self.is_empty():
            return self.items.pop(0)
        else:
            print("empty queue")
            return None

    def is_empty(self):
        return len(self.items) == 0

    @classmethod
    def get_queue_by_name(cls, name):
        """Returns a queue instance by its name."""
        return cls.all_queues.get(name)

class QueueOutOfRangeException(Exception):
    pass


In [11]:
try:
    q1 = QueueWithAttributes(name="queue1", size=2)
    q1.insert(1)
    q1.insert(2)
    q1.insert(3)  # Raises QueueOutOfRangeException

    q2 = QueueWithAttributes(name="queue2", size=1)
    q2.insert(1)
    q2.insert(2)  # Raises QueueOutOfRangeException

    q3 = QueueWithAttributes.get_queue_by_name("queue1")
    print(q3.pop())  # Output: 1
except QueueOutOfRangeException as e:
    print(f"Exception: {e}")


Exception: Queue size exceeded.


In [16]:
# Third Task Lab 5
import requests

class WeatherAPIClient:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "http://api.weatherapi.com/v1"

    def get_current_temperature(self, city):
        endpoint = "/current.json"
        params = {"key": self.api_key, "q": city}
        response = self._make_request(endpoint, params)
        return self._extract_current_temperature(response)

    def get_temperature_after(self, city, days, hour=None):
        endpoint = "/forecast.json"
        params = {"key": self.api_key, "q": city, "days": days}
        response = self._make_request(endpoint, params)
        return self._extract_temperature_after(response, days, hour)

    def get_lat_and_long(self, city):
        endpoint = "/search.json"
        params = {"key": self.api_key, "q": city}
        response = self._make_request(endpoint, params)
        return self._extract_lat_and_long(response)

    def _make_request(self, endpoint, params):
        try:
            url = f"{self.base_url}{endpoint}"
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error making API request: {e}")
            return {}

    def _extract_current_temperature(self, response):
        return response.get("current", {}).get("temp_c")

    def _extract_temperature_after(self, response, days, hour):
        forecast = response.get("forecast", {}).get("forecastday", [])
        if forecast:
            if hour is not None:
                for day in forecast:
                    for hour_data in day.get("hour", []):
                        if hour_data.get("time") == f"{days} {hour:02}:00":
                            return hour_data.get("temp_c")
            else:
                return forecast[0].get("day", {}).get("avgtemp_c")
        return None

    def _extract_lat_and_long(self, response):
        location = response[0] if response else {}
        return location.get("lat"), location.get("lon")

# Example usage:
api_key = "your_actual_weatherapi_com_api_key"
weather_client = WeatherAPIClient(api_key)

# Get current temperature
current_temp = weather_client.get_current_temperature("CityName")
print(f"Current Temperature: {current_temp}°C")

# Get temperature after 3 days
future_temp = weather_client.get_temperature_after("CityName", days=3)
print(f"Temperature after 3 days: {future_temp}°C")

# Get latitude and longitude
lat, lon = weather_client.get_lat_and_long("CityName")
print(f"Latitude: {lat}, Longitude: {lon}")

Error making API request: 403 Client Error: Forbidden for url: http://api.weatherapi.com/v1/current.json?key=your_actual_weatherapi_com_api_key&q=CityName
Current Temperature: None°C
Error making API request: 403 Client Error: Forbidden for url: http://api.weatherapi.com/v1/forecast.json?key=your_actual_weatherapi_com_api_key&q=CityName&days=3
Temperature after 3 days: None°C
Error making API request: 403 Client Error: Forbidden for url: http://api.weatherapi.com/v1/search.json?key=your_actual_weatherapi_com_api_key&q=CityName
Latitude: None, Longitude: None


In [2]:
import pickle

In [3]:
class Queue:
    def __init__(self):
        self.items = []

    def insert(self, value):
        self.items.append(value)

    def pop(self):
        if not self.is_empty():
            return self.items.pop(0)
        else:
            print("Warning: Cannot pop from an empty queue.")
            return None

    def is_empty(self):
        return len(self.items) == 0

In [4]:
class QueueWithAttributes(Queue):
    all_queues = {}

    def __init__(self, name, size):
        super().__init__()
        self.name = name
        self.size = size

        if self.name in self.all_queues:
            raise ValueError("Queue with this name already exists.")
        else:
            self.all_queues[self.name] = self

    def insert(self, value):
        if len(self.items) < self.size:
            super().insert(value)
        else:
            raise QueueOutOfRangeException("Queue size exceeded.")

    @classmethod
    def save(cls, filename):
        with open(filename, 'wb') as file:
            pickle.dump(cls.all_queues, file)

    @classmethod
    def load(cls, filename):
        with open(filename, 'rb') as file:
            cls.all_queues = pickle.load(file)


class QueueOutOfRangeException(Exception):
    pass

In [5]:
q = Queue()
q.insert(1)
q.insert(2)
print(q.pop()) 
print(q.pop())  
print(q.is_empty())  


try:
    q1 = QueueWithAttributes(name="queue1", size=2)
    q1.insert(1)
    q1.insert(2)
    q1.insert(3) 

    q2 = QueueWithAttributes(name="queue2", size=1)
    q2.insert(1)
    q2.insert(2)  

    
    QueueWithAttributes.save("queues.pkl")
    QueueWithAttributes.load("queues.pkl")

    q3 = QueueWithAttributes.all_queues.get("queue1")
    print(q3.pop())  # Output: 1
except QueueOutOfRangeException as e:
    print(f"Exception: {e}")


1
2
True
Exception: Queue size exceeded.
